# Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) data structure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on a trie;
3. Measure suggestion speed;

![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [1]:
!pip install pygtrie


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Build a trie upon a dataset

### 1.1. [5] Read the dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset. Pass the assert.

In [2]:
# after downloading the data, and unzipping it
import pandas as pd
data_file_name = 'user-ct-test-collection-01.txt'
# let's convert it to a tsv file 

with open(data_file_name, 'r') as r:
    with open('dataset.tsv', 'w') as w:
        w.write(r.read())


aol_data = pd.read_csv('dataset.tsv',sep='\t') 

assert aol_data.shape[0] == 3558411, "Dataset size does not match"

In [3]:
aol_data.columns = ['id', 'query', 'time', 'rank', 'url']
aol_data[~aol_data['rank'].isna()].head(10)

# the data is loaded correctly..

,id,query,time,rank,url
6,142,westchester.gov,2006-03-20 03:55:57,1.0,http://www.westchestergov.com
14,142,207 ad2d 530,2006-04-08 01:31:14,1.0,http://www.courts.state.ny.us
17,142,vera.org,2006-04-08 08:38:42,1.0,http://www.vera.org
27,217,lottery,2006-03-01 11:58:51,1.0,http://www.calottery.com
28,217,lottery,2006-03-01 11:58:51,1.0,http://www.calottery.com
29,217,ameriprise.com,2006-03-01 14:06:23,1.0,http://www.ameriprise.com
32,217,mizuno.com,2006-03-07 22:41:17,1.0,http://www.mizuno.com
35,217,asiansexygoddess.com,2006-03-16 14:31:36,1.0,http://www.asiansexygoddess.com
37,217,bestasiancompany.com,2006-03-20 15:15:43,1.0,http://www.bestasiancompany.com
38,217,lottery,2006-03-27 14:10:38,1.0,http://www.calottery.com


### 1.2. [10] Build a Trie

We want a suggest function to be **non-sensitive to stop words** because we don't want to upset the users if they confuses/omits prepositions. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build a Trie based on the dataset, **storing query statistics such as query _frequency_, urls and ranks in the nodes**. Some queries may have no associated urls, others may have multiple ranked urls. Think of the way to store this information.

Pass the asserts.

In [4]:
import pygtrie
aol_trie = pygtrie.CharTrie()

In [5]:
# each query should be associated with urls, ranks of these urls and also the frequency
stops = set('a on at of to is from for and with using the in &'.split())

# we will remove the stop words from our representations of the queries
import re

def process_query(query: str):
    return " ".join([x for x in re.split(r'\s+', query.lower().strip()) if x not in stops])

print(process_query('I am at my place. He is on the roof'))

import numpy as np
import itertools

class QueryNode:
    # this class represents a query in the trie
    # it stores all the necessary information about the query
    def __init__(self, query):
        # save the query
        self.query = query
        self.freq = 0
        # the urls associated with the queries
        self.urls = []
        # the ranks of each of the urls
        self.ranks = []
    
    def add_url(self, url, rank):
        # first add the url to the list
        self.urls.append(url)
        self.ranks.append(rank)
        self.freq += 1
    
    def best_url(self):
        # returns the url with the highest rank
        return self.urls[np.argmin(self.ranks)] # return the url with the highest rank
    
class QueryTrieNode:
    # each concrete query should be associated with a reduced processed version
    # all the trie should store the corrected ones (without stop words mainly)

    def __init__(self):
        self.queries = []
        self.queries_dict = {}

    def add_query(self, original_query, url:str, rank:float):

        if original_query in self.queries_dict:
            # extract the index to access directly
            index_query = self.queries_dict[original_query]
            self.queries[index_query].add_url(url, rank)
            return 
        # if this case the query was seen for the 1st time
        self.queries_dict[original_query] = len(self.queries)
        # add it to the queries                     
        self.queries.append(QueryNode(original_query))
        # add the url and the rank to the last query
        self.queries[-1].add_url(url, rank)
    @property
    def urls(self):
        all_urls = [q.urls for q in self.queries]
        return list(itertools.chain(*all_urls))

# create a function to add a query to the trie data structure


def add_query_to_trie(row: str, trie:pygtrie=None):
    if trie is None:
        trie = aol_trie
    # extract the original query
    original_query = row['query']
    # the original query might be null 
    if isinstance(original_query, str):
        # process it
        query = process_query(original_query)
        if trie.has_node(query) != pygtrie.Trie.HAS_VALUE:
            trie[query] = QueryTrieNode()
        trie[query].add_query(original_query, row['url'], row['rank'])


i am my place. he roof


In [6]:
# save the tree to avoid building it on multiple occasions
import pickle
aol_trie

# file to pickle the trie instead of building it with every run
file_name = 'aol_trie.pkl'


# with open(file_name, 'wb') as file:
#     pickle.dump(aol_trie, file)
#     print(f'Object successfully saved to "{file_name}"')

import os
if os.path.exists(file_name):
    # loading the trie
    with open(file_name, 'rb') as f:
        aol_trie = pickle.load(f)
else:
    # build the trie by applyinh the add_query_to_trie function
    # to each row in the dataset
    aol_data.apply(add_query_to_trie,axis=1)



In [7]:
# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a property of list type. But you can do something different. 
    bag += val.urls
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ <__main__.QueryTrieNode object at 0x0000019E770218B0>
sample questions immigration interview ~ <__main__.QueryTrieNode object at 0x0000019E77021970>
sample questions interview ~ <__main__.QueryTrieNode object at 0x0000019E77021A30>
sample questions family interview ~ <__main__.QueryTrieNode object at 0x0000019E77021AF0>
sample questions sociology race ethnicity ~ <__main__.QueryTrieNode object at 0x0000019E77021BB0>
sample questions biology ~ <__main__.QueryTrieNode object at 0x0000019E77021C70>
sample questions us citizenship test ~ <__main__.QueryTrieNode object at 0x0000019E77021D30>
sample questionarie teaching evaluation ~ <__main__.QueryTrieNode object at 0x0000019E77021DF0>
sample questionnaire teaching evaluation ~ <__main__.QueryTrieNode object at 0x0000019E77021EB0>
sample questionnaire clinical research coordinators certification ~ <__main__.QueryTrieNode object at 0x0000019E77021F70>


## 2. [15] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Pass the asserts.

Question for analysis: What is the empirical threshold for minimal prefix for suggest?

In [8]:
def complete_user_query(query: str, trie, top_k=5, print_urls:bool=False) -> list[str]:
    # first make sure to extract the processed request
    p_query = process_query(query.strip().lower())

    # firt check if the shortened version of the query was seen before
    if not trie.has_subtrie(p_query):
        return []

    # the query given might not represent a complete query so we need to extract any query from our database that might correspond to user's input
    possible_queries = [val.queries for _, val in trie.iteritems(p_query)]            
    # flatten it
    possible_queries = itertools.chain(*possible_queries)
    # sort by frequency
    possible_queries = sorted(possible_queries, key=lambda x : x.freq, reverse=True)

    result = [pq.query for pq in possible_queries[:top_k]] 

    # display the best url for the top_k queries
    if print_urls:
        for pq in possible_queries[:top_k]:
            best_url = pq.best_url()
            if isinstance(best_url, str):
                print(best_url)
            else:
                print(f'no url for query: {pq.query}')

    return result

In [9]:
inp = "trie"
print("Query:", inp)
print("Results:")
res = complete_user_query(inp, aol_trie, print_urls=True)
print(res)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

assert "boys and girls club of conyers georgia" \
            in complete_user_query("boys girls club conyers", aol_trie, 10), "Should be here"

Query: trie
Results:
http://www.triedntruetattoo.com
no url for query: triest
http://avalon.unomaha.edu
no url for query: tried and failed
no url for query: tried and truechildren's consignment sale
['tried and true tattoo', 'triest', 'triethanalomine', 'tried and failed', "tried and truechildren's consignment sale"]


## 3. Measure suggest speed ##

### 3.1. [10] Full Trie test

Check how fast your search is working. Consider changing your code if it takes too long on average.

Sucess criterion:
- there is an average and a standard deviation for **multiple runs** of the given bucket.
- there is an average and a standard deviation for **multiple runs** of naive search in the unindexed dataset.

### EXECUTION TIME: INDEXED SEARCH

In [10]:
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

# measure avg execution time (in milliseconds) per query and print it out
print("Time per query with the index data structure : \n")

# let's try to save the results of the runs
run_results = []
# run_results = [%timeit -o complete_user_query(q, aol_trie, 1) for q in inp_queries]
for query in inp_queries:
    run = %timeit -o complete_user_query(query, aol_trie, 1)
    run_results.append(run.all_runs[:10]) # saving the execution time (not the object itself)

Time per query with the index data structure : 

27.2 ms ± 843 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
39.9 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.01 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
90.3 µs ± 3.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
40 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
357 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
23.8 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
38 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
35.4 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
26.8 µs ± 2.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
2.3 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
units = [0.1, 10 ** -4, 10 ** -4, 10 ** -4, 10 ** -4, 0.1, 0.1, 10 ** -4, 0.1, 10 ** -4, 10 ** -4]
runs_t = [list(map(lambda x: x * u, run)) for run, u in zip(run_results, units)]
results = pd.DataFrame(data=dict(zip(inp_queries, runs_t)))
# the table aboves saves the results of the first 10 runs of the indexed search for each query
# let's calculate the variance and mean
def add_stats(row):
    row['std'] = np.std(row)
    row['mean'] = np.mean(row)
    return row 

r = results.T.apply(add_stats, axis=1).loc[:, ['std', 'mean']].T
r

,inf,the best,information retrieval,sherlock hol,carnegie mell,babies r,new york,googol,inter,USA sta,Barbara
std,0.000843,0.000002,0.000002,0.000004,0.000002,0.001258,0.001172,0.000001,0.002985,0.000002,0.000002
mean,0.023913,0.000035,0.000079,0.000079,0.000035,0.031364,0.021000,0.000033,0.031335,0.000024,0.000020


The table above displays both the mean and the standard deviation of the execution time accross 10 runs for the indexed search function:
* for the query *inf*, the indexed search function takes 0.023 seconds (23 milliseconds) on average (across 10 runs) with a standard deviation of only 0.0008 we can expect the execution time to be within (21 milliseconds and 25 milliseconds)  

* for the query *inter*, the indexed search function takes 0.023 seconds (23 milliseconds) on average (across 10 runs) with a standard deviation of only 0.0008 we can expect the execution time to be within (21 milliseconds and 25 milliseconds)  

* The execution time for queries unseen in the training (e.g information retrieval) data is quite small as the trie index enable to quickly determine such queries without extra overhead.  

### EXECUTION TIME: UNINDEXED SEARCH (NAIF)

In [12]:
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]


# let's rewrite the code to use the dataset directly without the trie index
def complete_user_query_unindexed(query, dataset, top_k=5):
    freq = {}
    for _, row in dataset.loc[dataset['query'].str.startswith(query, na=False)].iterrows():
        if type(row['query']) == str:
            if freq.get(row['query']) is None:
                freq[row['query']] = 0
            freq[row['query']] += 1
    return sorted(freq.keys(), reverse=True, key=lambda x: freq[x])[:top_k]

In [13]:
print("Time per query (Unindexed dataset): \n")

# let's try to save the results of the runs
run_results = []
# run_results = [%timeit -o complete_user_query(q, aol_trie, 1) for q in inp_queries]
for query in inp_queries:
    run = %timeit -o complete_user_query_unindexed(query, aol_data, 1)
    run_results.append(run.all_runs[:10]) # saving the execution time for the first 10 runs (not the object itself)


Time per query (Unindexed dataset): 

1.81 s ± 47.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.63 s ± 52.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.52 s ± 25.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.5 s ± 26.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.52 s ± 37.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.55 s ± 40.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.85 s ± 46.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.54 s ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.18 s ± 48.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.46 s ± 19.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.5 s ± 42.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
results = pd.DataFrame(data=dict(zip(inp_queries, run_results)))
# the table aboves saves the results of the first 10 runs of the indexed search for each query
# let's calculate the variance and mean
def add_stats(row):
    row['std'] = np.std(row)
    row['mean'] = np.mean(row)
    return row 

r = results.T.apply(add_stats, axis=1).loc[:, ['std', 'mean']].T
r

,inf,the best,information retrieval,sherlock hol,carnegie mell,babies r,new york,googol,inter,USA sta,Barbara
std,0.047106,0.052809,0.025594,0.026308,0.037463,0.040792,0.046166,0.033002,0.048356,0.019132,0.042546
mean,1.588572,1.435721,1.336359,1.319440,1.336617,1.365361,1.624975,1.347300,1.917363,1.283905,1.317676


The table above displays both the mean and the standard deviation of the execution time accross 10 runs for the naif unindexed search function:
* for the query *inf*, the indexed search function takes 1.58 seconds on average (across 10 runs) with a 0.047 standard deviation. We can expect the execution time to be within (1.73 seconds and 1.43 seconds).  
Notice that the performance is more than 7 times slower than the indexed search function

* for the query *inter*, the indexed search function takes 1.91 seconds on average (across 10 runs) with a 0.048 standard deviation. We can expect the execution time to be within (2.1 seconds and 1.7 seconds).  
Notice that the performance is more than 7 times slower than the indexed search function

* The execution time between the different queries is not so significant as the naif method ends up scanning most of the dataset regardless of whether the query was seen before or not.

## 4. [10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

You can use use [pyspellchecker](https://pypi.org/project/pyspellchecker/) `candidates()` call, or use any other spellchecker implementation.

In [15]:
# Install the spell checker
!pip install -U pyspellchecker


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from spellchecker import SpellChecker
spell = SpellChecker()
# spell.word_frequency.load_words(['mell'])
# first of all let's create a dictionary of the words and their frequencies in our dataset
from collections import Counter
word_freq = Counter()

punc = r'[#%&\(\)\*\+,\/:;<=>@\[\\\]\^_`{\|}~\$-]+'

# this function is needed for cleaning the text
def process_text(text):
    if text is None: 
        return
    
    # a minimal data processing
    text = text.lower().strip()
    # remove any non desirable characters: 
    text = re.sub(punc, '', text)

    # remove any segment of text where there is not alphabetic characters within alphabetic characters
    regex = r'(\w+[^a-zA-Z1-9\s]+)*(\w+[^a-zA-Z1-9\s]+\w+)'    
    text = re.sub(regex, ' ', text)

    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    if len(text) > 0:     
        return text
    
    return 

t = 'thi**s t@@ex~t is full of extra[] characters, it need##s to b%e cleaned__'
print(process_text(t))

this text is full of extra characters it needs to be cleaned


In [22]:
def store_word_frequency(text):
    """
    Counts the frequency of each word present in the dataset
    """
    if not isinstance(text, str):
        return  
    text = process_text(text)
    if text: # the text might be None
        words = text.lower().strip().split()
        word_freq.update(dict(zip(words, [1 for _ in words])))
    

aol_data.apply(lambda row: store_word_frequency(row["query"]), axis=1)
word_freq
# we can see here the dictionary is relatively clean (with minimal extra characters and urls...)

Counter({'dfdf': 2,
         'ad2d': 2,
         '530': 32,
         'merit': 93,
         'release': 678,
         'appearance': 41,
         'lottery': 6174,
         'susheme': 1,
         'p': 1128,
         '.': 18900,
         "p'": 5,
         "'": 1646,
         'buddylis': 1,
         'vietnam': 705,
         'googl': 132,
         'ozark': 111,
         'horse': 3486,
         'blankets': 164,
         'gall': 115,
         'stones': 553,
         'gallstones': 15,
         'http': 25004,
         'photos': 6972,
         '88145967': 4,
         '24368586': 4,
         'in': 90091,
         'pool32148876': 4,
         'href': 380,
         'a': 33791,
         'alt': 238,
         'files': 786,
         'dellcomputers': 5,
         'cascadefamilymedical': 1,
         'pop': 652,
         'up': 4996,
         'adds': 99,
         'the': 77080,
         'childs': 85,
         'wonderland': 121,
         'company': 3824,
         'grand': 3123,
         'rapids': 380,
         '

In [23]:
# the main idea is  to take the suggestions of the spell checker 
# and choose the ones with the highest frequency in the vocabulary built out of the queries dataset

def most_likely_candidate(words):
    # sort by the frequency of the word in the vocabulary (set to 0 if the word does not appear)
    return sorted(words, reverse=True, key=lambda x: word_freq.get(x) or 0)[0]
def complete_user_query_with_spellchecker(query, trie, top_k=5):
    global spell
    query_words = query.lower().strip().split()
    corrected_words = [most_likely_candidate(spell.candidates(word)) if len(word)> 2 else word for word in query.split()]
    for i in range(len(corrected_words)):
        if trie.has_subtrie(" ".join(query_words)):
            # in this case we proceed further to understand whether we need to modify the query or not
            modified_query = query_words.copy()
            modified_query[i] = corrected_words[i]
            
            if not trie.has_subtrie(" ".join(modified_query)): # in case the modified query is actually not in the trie
                continue
        
            node_modified = list(trie.iteritems(" ".join(modified_query)))
            node_original = list(trie.iteritems(" ".join(query_words)))

            if len(node_modified) < len(node_original): # the original node is sitll more likely 
                continue # keep the original 

        query_words[i] = corrected_words[i]
    
    print(" ".join(query_words))
    return complete_user_query(" ".join(query_words), trie, top_k)    

In [24]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]

for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, aol_trie, 5) == \
            complete_user_query_with_spellchecker(q, aol_trie, 5), "Assert {} and {} give different results".format(q, qc)

information retrieval
sherlock hol
carnegie mell
babies r
barbara
